# Preparation of biased coexsistence simulations

In [1]:
import numpy as np, sys, os, glob
import matplotlib.pylab as plt
import MDAnalysis

/Users/sigbjobo/miniforge3/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [2]:
standard_sim=os.path.abspath('template/')

press_box = np.loadtxt('../2-Bulk/avg_IceVI_box.dat')
presures=np.unique(press_box[:,0])

In [3]:
sims = dict()
for a in press_box[:]:
    fold='COEX_{}atm/{}K/'.format(a[0],a[1])
    os.system('mkdir -p {}'.format(fold))
    os.system('cp ../../../2-GenerateTrainingset/GraphFiles/Iteration1/graph-compress.pb {}/../graph.pb'.format(fold))
    os.system('cp {}/* {}'.format(standard_sim,fold))
    sims[fold] = dict()
    p=a[0]
    t=a[1]
    
    cmd ='sed -i  \"s#REPLACE_PRESSURE#{}#g\" {}/in.pressure\n'.format(p*1.01325,fold)
    cmd+='sed -i  \"s#REPLACE_TEMP#{}#g\" {}/in.temp\n'.format(t,fold)
    cmd+='sed -i  \"s#REPLACE_TEMP#{}#g\" {}/plumed*.dat'.format(t,fold)
    os.system(cmd)
    u = MDAnalysis.Universe(standard_sim+'/water.data', in_memory=True)
    
    scaled_coord=MDAnalysis.lib.distances.transform_RtoS(u.atoms.positions,u.dimensions)
   
    print('before',u.dimensions)
    dimensions_new=a[2:]
    max_l=np.max(u.dimensions[:3])
    max_dim=np.where(u.dimensions==max_l)[0][0]
    dimensions_new[max_dim]=u.dimensions[max_dim]
    u.dimensions=dimensions_new
    u.atoms.positions=MDAnalysis.lib.distances.transform_StoR(scaled_coord,u.dimensions)
    print('after',u.dimensions)

    u.atoms.write(fold+'/water.data')

sed: 1: "COEX_8000.0atm/25.057K/ ...": invalid command code C
sed: 1: "COEX_8000.0atm/25.057K/ ...": invalid command code C
sed: 1: "COEX_8000.0atm/25.057K/ ...": invalid command code C
/Users/sigbjobo/miniforge3/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


before [52.789078 24.971228 23.019014 90.       90.       90.      ]


ValueError: could not broadcast input array from shape (5,) into shape (6,)